In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import requests

app = Flask(__name__)

# 軽量CNNモデルの読み込み
model = tf.keras.models.load_model('model/speech_commands_cnn.h5')

COMMANDS = ['left', 'right', 'forward', 'stop', 'go']
NEXTJS_LOG_URL = "https://your-nextjs-app.vercel.app/api/log"

@app.route('/recognize', methods=['POST'])
def recognize_audio():
    audio_data = request.files['audio']

    # 前処理
    processed_audio = preprocess_audio(audio_data)

    # CNN推論
    prediction = model.predict(processed_audio)
    command = COMMANDS[np.rgmax(prediction)]
    confidence = float(np.max(prediction))

    # Next.jsにログ送信
    log_data = {
        'command': command,
        'confidence': confidence,
        'timestamp': time.time()
    }
    requestspost(NEXTJS_LOG_URL, json=log_data)

    return jsonify({
        'command': command,
        'confidence': confidence
    })

def preprocess_audio(audio_file):
    # MFCC特徴量抽出など
    pass

# Colabで実行
app.run(host='0.0.0.0', port=5000)


: 

In [ ]:
def create_speech_commands_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(COMMANDS), activation='softmax')
    ])
    return model
